In [2]:
import numpy as np
import datetime as dt
import pandas as pd

In [4]:
# IMPORTANDO A BASE
base = pd.read_csv('Criando uma apresentação executiva.csv')
base.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94


In [10]:
# AJUSTANDO AS DATA QUE VAMOS UTILIZAR

# Mudando data de string para date
base['Order Date'] = pd.to_datetime(base['Order Date'], format='%d/%m/%Y')

# Criando colunas mês e ano
base['Ano'] = base['Order Date'].dt.year
base['Mes'] = base['Order Date'].dt.month

base.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Ano,Mes
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2017,11
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,2017,11


In [18]:
# PEGANDO SOMENTE AS COLUNAS QUE VAMOS PRECISAR USAR
baseresumida = base[['Product Name','Mes','Ano','Sales']]
baseresumida.head(2)

,Product Name,Mes,Ano,Sales
0,Bush Somerset Collection Bookcase,11,2017,261.96
1,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",11,2017,731.94


# Comparando a venda ano a ano

In [31]:
# AGRUPANDO PRODUTOS POR ANO
produtoano = baseresumida.groupby(['Product Name','Ano'])['Sales'].sum()

# Transformando em um DataFrame
produtoano = produtoano.reset_index()

# Mandando exibir
produtoano

,Product Name,Ano,Sales
0,"""While you Were Out"" Message Book, One Form pe...",2018,25.228
1,"#10 Gummed Flap White Envelopes, 100/Box",2016,6.608
2,"#10 Gummed Flap White Envelopes, 100/Box",2017,24.780
3,"#10 Gummed Flap White Envelopes, 100/Box",2018,9.912
4,#10 Self-Seal White Envelopes,2017,86.502
...,...,...,...
5269,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,2018,205.086
5270,netTALK DUO VoIP Telephone Service,2015,335.936
5271,netTALK DUO VoIP Telephone Service,2016,230.956
5272,netTALK DUO VoIP Telephone Service,2017,377.928


In [37]:
# FILTRANDO APENAS 1 PRODUTO
base2=produtoano[produtoano['Product Name'] == 'netTALK DUO VoIP Telephone Service']
base2

,Product Name,Ano,Sales
5270,netTALK DUO VoIP Telephone Service,2015,335.936
5271,netTALK DUO VoIP Telephone Service,2016,230.956
5272,netTALK DUO VoIP Telephone Service,2017,377.928
5273,netTALK DUO VoIP Telephone Service,2018,167.968


Vamos utilizar o .pivot() para transformar os anos em colunas
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html

In [56]:
# UTILIZANDO O .PIVOT() PARA CRIAR UMA NOVA TABELA

# Criando uma nova base onde os anos são colunas e usando o reset_index para virar um DataFrame
basenova = produtoano.pivot(index='Product Name',columns='Ano',values='Sales').reset_index()

# Renomeando as colunas para que os anos sejam strings
basenova.columns= ('Product Name','2015','2016','2017','2018')

# Exibindo base
basenova

,Product Name,2015,2016,2017,2018
0,"""While you Were Out"" Message Book, One Form pe...",NaN,NaN,NaN,25.228
1,"#10 Gummed Flap White Envelopes, 100/Box",NaN,6.608,24.780,9.912
2,#10 Self-Seal White Envelopes,NaN,NaN,86.502,22.180
3,"#10 White Business Envelopes,4 1/8 x 9 1/2",156.700,12.536,125.360,84.618
4,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",62.928,134.596,NaN,89.148
...,...,...,...,...,...
1844,iKross Bluetooth Portable Keyboard + Cell Phon...,NaN,251.400,159.220,67.040
1845,iOttie HLCRIO102 Car Mount,39.980,NaN,55.972,119.940
1846,iOttie XL Car Mount,NaN,NaN,NaN,223.888
1847,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,32.382,43.176,161.910,205.086


In [57]:
# CRIANDO UMA NOVA COLUNA COM A DIFERENÇA DE VENDAS ENTRE 2018 E 2017
basenova['Dif18_17'] = basenova['2018'] - basenova['2017']
basenova

,Product Name,2015,2016,2017,2018,Dif18_17
0,"""While you Were Out"" Message Book, One Form pe...",NaN,NaN,NaN,25.228,NaN
1,"#10 Gummed Flap White Envelopes, 100/Box",NaN,6.608,24.780,9.912,-14.868
2,#10 Self-Seal White Envelopes,NaN,NaN,86.502,22.180,-64.322
3,"#10 White Business Envelopes,4 1/8 x 9 1/2",156.700,12.536,125.360,84.618,-40.742
4,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",62.928,134.596,NaN,89.148,NaN
...,...,...,...,...,...,...
1844,iKross Bluetooth Portable Keyboard + Cell Phon...,NaN,251.400,159.220,67.040,-92.180
1845,iOttie HLCRIO102 Car Mount,39.980,NaN,55.972,119.940,63.968
1846,iOttie XL Car Mount,NaN,NaN,NaN,223.888,NaN
1847,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,32.382,43.176,161.910,205.086,43.176


In [58]:
# CRIANDO UMA NOVA COLUNA COM O % DA DIFERENÇA
basenova['%Dif18_17'] = basenova['2018']/basenova['2017']
basenova

,Product Name,2015,2016,2017,2018,Dif18_17,%Dif18_17
0,"""While you Were Out"" Message Book, One Form pe...",NaN,NaN,NaN,25.228,NaN,NaN
1,"#10 Gummed Flap White Envelopes, 100/Box",NaN,6.608,24.780,9.912,-14.868,0.400000
2,#10 Self-Seal White Envelopes,NaN,NaN,86.502,22.180,-64.322,0.256410
3,"#10 White Business Envelopes,4 1/8 x 9 1/2",156.700,12.536,125.360,84.618,-40.742,0.675000
4,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",62.928,134.596,NaN,89.148,NaN,NaN
...,...,...,...,...,...,...,...
1844,iKross Bluetooth Portable Keyboard + Cell Phon...,NaN,251.400,159.220,67.040,-92.180,0.421053
1845,iOttie HLCRIO102 Car Mount,39.980,NaN,55.972,119.940,63.968,2.142857
1846,iOttie XL Car Mount,NaN,NaN,NaN,223.888,NaN,NaN
1847,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,32.382,43.176,161.910,205.086,43.176,1.266667


In [62]:
# ORDENANDO OS VALORES DA BASE
basenova=basenova.sort_values('Dif18_17')
basenova

,Product Name,2015,2016,2017,2018,Dif18_17,%Dif18_17
687,GBC Ibimaster 500 Manual ProClick Binding System,NaN,5402.958,12860.562,760.980,-12099.582,0.059172
838,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",2415.402,NaN,6367.878,1854.248,-4513.630,0.291188
19,"3D Systems Cube Printer, 2nd Generation, Magenta",NaN,NaN,9099.930,5199.960,-3899.970,0.571429
803,Hewlett Packard 610 Color Digital Copier / Pri...,1999.960,1799.964,4199.916,899.982,-3299.934,0.214286
498,DMI Eclipse Executive Suite Bookcases,4809.408,2429.753,3406.664,400.784,-3005.880,0.117647
...,...,...,...,...,...,...,...
1834,Xiaomi Mi3,NaN,NaN,2279.960,NaN,NaN,NaN
1837,Zebra GK420t Direct Thermal/Thermal Transfer P...,NaN,NaN,703.710,NaN,NaN,NaN
1838,Zebra GX420t Direct Thermal/Thermal Transfer P...,NaN,5521.880,265.475,NaN,NaN,NaN
1839,Zebra ZM400 Thermal Label Printer,NaN,6965.700,NaN,NaN,NaN,NaN


In [63]:
# Selecionando apenas as 20 primeiras linhas
basenova.head(20)

,Product Name,2015,2016,2017,2018,Dif18_17,%Dif18_17
687,GBC Ibimaster 500 Manual ProClick Binding System,NaN,5402.9580,12860.562,760.980,-12099.582,0.059172
838,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",2415.402,NaN,6367.878,1854.248,-4513.630,0.291188
19,"3D Systems Cube Printer, 2nd Generation, Magenta",NaN,NaN,9099.930,5199.960,-3899.970,0.571429
803,Hewlett Packard 610 Color Digital Copier / Pri...,1999.960,1799.9640,4199.916,899.982,-3299.934,0.214286
498,DMI Eclipse Executive Suite Bookcases,4809.408,2429.7530,3406.664,400.784,-3005.880,0.117647
155,Apple iPhone 5,5328.606,NaN,5198.640,2469.354,-2729.286,0.475000
894,Ibico EPK-21 Electric Binding System,9449.950,NaN,4535.976,1889.990,-2645.986,0.416667
428,Chromcraft Round Conference Tables,NaN,244.0060,5176.413,2788.640,-2387.773,0.538721
369,Bretford “Just In Time” Height-Adjustable Mult...,NaN,NaN,3965.300,1669.600,-2295.700,0.421053
1024,Logitech P710e Mobile Speakerphone,772.470,617.9760,5407.290,3398.868,-2008.422,0.628571


In [64]:
# QUANTOS PRODUTOS TIVERAM AUMENTO DE VENDA? EM VALOR
basenova[basenova['Dif18_17']>0]

,Product Name,2015,2016,2017,2018,Dif18_17,%Dif18_17
1128,Newell 322,22.568,NaN,3.640,4.368,0.728,1.200000
254,Avery Hi-Liter Smear-Safe Highlighters,NaN,54.896,29.200,30.368,1.168,1.040000
1795,Xerox 201,NaN,25.920,41.472,42.768,1.296,1.031250
675,G.E. Halogen Desk Lamp Bulbs,36.296,13.960,5.584,6.980,1.396,1.250000
1623,Wilson Jones Active Use Binders,29.120,43.680,20.384,21.840,1.456,1.071429
...,...,...,...,...,...,...,...
112,Adjustable Depth Letter/Legal Cart,362.920,1959.768,362.920,4754.252,4391.332,13.100000
649,Fellowes PB500 Electric Punch Plastic Comb Bin...,2033.584,15760.276,2287.782,7371.742,5083.960,3.222222
1387,Samsung Galaxy Mega 6.3,1679.960,NaN,3023.928,9239.780,6215.852,3.055556
685,GBC DocuBind TL300 Electric Binding System,5740.736,1345.485,1793.980,10943.278,9149.298,6.100000


In [65]:
# QUANTOS PRODUTOS TIVERAM QUEDA DE VENDA? EM VALOR
basenova[basenova['Dif18_17']<0]

,Product Name,2015,2016,2017,2018,Dif18_17,%Dif18_17
687,GBC Ibimaster 500 Manual ProClick Binding System,NaN,5402.958,12860.562,760.980,-12099.582,0.059172
838,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",2415.402,NaN,6367.878,1854.248,-4513.630,0.291188
19,"3D Systems Cube Printer, 2nd Generation, Magenta",NaN,NaN,9099.930,5199.960,-3899.970,0.571429
803,Hewlett Packard 610 Color Digital Copier / Pri...,1999.960,1799.964,4199.916,899.982,-3299.934,0.214286
498,DMI Eclipse Executive Suite Bookcases,4809.408,2429.753,3406.664,400.784,-3005.880,0.117647
...,...,...,...,...,...,...,...
92,Acme Kleen Earth Office Shears,30.264,NaN,7.760,6.208,-1.552,0.800000
16,3-ring staple pack,5.640,16.920,10.528,9.024,-1.504,0.857143
59,Acco Economy Flexible Poly Round Ring Binder,1.044,NaN,4.176,3.132,-1.044,0.750000
1480,SpineVue Locking Slant-D Ring Binders by Cardinal,15.538,8.226,43.872,42.958,-0.914,0.979167


In [67]:
# QUANTOS PRODUTOS NAO TIVERAM QUEDA/AUMENTO DE VENDA? EM VALOR
basenova[basenova['Dif18_17']==0]

,Product Name,2015,2016,2017,2018,Dif18_17,%Dif18_17
1229,Panasonic KP-350BK Electric Pencil Sharpener w...,NaN,55.328,34.580,34.580,0.0,1.0
371,Brother DCP1000 Digital 3 in 1 Multifunction M...,1199.960,2699.910,479.984,479.984,0.0,1.0
504,Dana Swing-Arm Lamps,25.632,49.128,17.088,17.088,0.0,1.0
1821,Xerox 225,45.360,19.440,66.096,66.096,0.0,1.0
1566,Tops Green Bar Computer Printout Paper,48.940,NaN,146.820,146.820,0.0,1.0
392,C-Line Cubicle Keepers Polyproplyene Holder w/...,164.220,131.376,109.480,109.480,0.0,1.0
207,Avery 499,82.668,23.904,34.860,34.860,0.0,1.0
1311,Project Tote Personal File,67.344,98.210,14.030,14.030,0.0,1.0
1144,Newell 337,NaN,NaN,5.248,5.248,0.0,1.0
110,"Adams Telephone Message Books, 5 1/4” x 11”",4.832,56.776,9.664,9.664,0.0,1.0
